In [ ]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
from hs2 import herdingspikes
from probe import NeuralProbe
import numpy as np

%matplotlib inline

In [ ]:
data_path = 'data/rawDataSample.bin'
# data = h5py.File(data_path, 'r')
# data = np.fromfile(data_path, dtype=np.int16).reshape((1800000,385))

H = herdingspikes()

In [ ]:
# detectData(data, neighbours, spikefilename, shapefilename, channels, sfd, thres, maa = None, maxsl = None, minsl = None, ahpthr = None, tpre = 1, tpost = 2)
# MinAvgAmp minimal avg. amplitude of peak (in units of Qd)
# MaxSl dead time in frames after peak, used for further testing
# MinSl length considered for determining avg. spike amplitude
# AHPthr signal should go below that threshold within MaxSl-Slmin frames

NeuroPixel = NeuralProbe(num_channels = 385,
                         spike_delay = 5,
                         spike_peak_duration = 5,
                         noise_duration = 2,
                         noise_amp = 80000,
                         fps = 30000,
                         positions_path="positions",
                         neighbors_path="neighbormatrix")

to_localize = True
cutout_length = 40
threshold = 20 #20

In [ ]:
NeuroPixel.Show(figwidth=2, showNeighbors=[-20])

In [ ]:
H.DetectFromRaw(data_path, NeuroPixel,
                to_localize, cutout_length, threshold,
                maa=0, maxsl=12, minsl=3, ahpthr=0)

# # OR

H.LoadDetected(NeuroPixel)

In [ ]:
H.PlotTracesChannels(data_path, 1008)

In [ ]:
plt.figure(figsize=(10, 4))
H.PlotAll(invert=True, s=2, alpha=0.5)

In [ ]:
H.CombinedClustering(alpha=40,
                    bandwidth = 20, bin_seeding=True, min_bin_freq=10,
                    pca_ncomponents=2, pca_whiten=True,
                    n_jobs=-1)

In [ ]:
plt.figure(figsize=(10, 5))
H.PlotAll(invert=True, s=10)
plt.xlim((1350,1600))
plt.title("MeanShift, bandwidth=20, min_bin_freq=10")

In [ ]:
from sklearn.cluster import DBSCAN
H.CombinedClustering(eps=8, alpha=40, clustering_algorithm=DBSCAN,
                    min_samples=5, pca_ncomponents=2, pca_whiten=True,
                    n_jobs=-1)

In [ ]:
plt.figure(figsize=(10, 5))
H.PlotAll(invert=True, s=10)
plt.title("DBSCAN eps=8, min_samples=5")
plt.xlim((1350,1600))


In [ ]:
units = (121,4)

H.PlotShapes(units)